# Applications in NLP:
*Translation
*Information Extraction
*Summarization
*Parsing
*Question Answering
*Sentiment Analysis
*Text Classification
And many more…

In [ ]:
# import nltk
import numpy as np
import pandas as pd
import nltk 
from tensorflow.python.keras.preprocessing.text import Tokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.isri import ISRIStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import re,string

# Read Our Text File

In [ ]:
#fp = open('BrainTracy.txt') # Open file on read mode


In [ ]:
#fp.read()

In [ ]:
#text1 = fp.read().split(".") 
text1 = 'He is the king . The king is royal . She is the royal queen'

# Make Sure We already uploaded our text file

In [ ]:
text1[:5]  

In [ ]:
text2=str(text1)

In [ ]:
text2

## Punctuation Removal

In [ ]:
text2=re.sub(r"[^0-9A-Za-z]"," ",text2)           #sub	Replaces one or many matches with a string

In [ ]:
text2

# Text Normalization

In [ ]:
text3=text2.lower()

In [ ]:
text3

# Tokenizer 

In [ ]:
text3=text3.split()    

In [ ]:
text3

In [ ]:
tokenizer=Tokenizer(num_words=400)

In [ ]:
text4=tokenizer.fit_on_texts(text3)

In [ ]:
tokenizer.word_index

In [ ]:

#tokenz=tokenizer.fit_on_sequences(text2)

In [ ]:
stop_words

# Remove Stop Words

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')


#stopWord = pd.read_excel('En_stops.xlsx',header=None,index_col=False)

In [ ]:
#stopWord1=list(stopWord[0])

In [ ]:
stop_words1 = stopwords.words('english')+ list(string.punctuation)


In [ ]:
stop_words1

In [ ]:
tokenizer1=Tokenizer(num_words=400)
text6=tokenizer1.fit_on_texts(text5)

In [ ]:
tokenizer1.word_index

# lemmatization

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
nltk.download('wordnet')
lemmed =[WordNetLemmatizer().lemmatize(w) for w in text5]

In [ ]:
lemmed

# Text Tagging

In [ ]:
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')

In [ ]:
pos_tag(text5)

In [ ]:
from nltk.tokenize import word_tokenize,wordpunct_tokenize

In [ ]:
nltk.download('punkt')
text7=str(text1)
text8=word_tokenize(text7)

In [ ]:
text8

# Bag of Words

# Shortcoming of Bag of Words method


It ignores the order of the word, for example, this is bad = bad is this.
It ignores the context of words. Suppose If I write the sentence "He loved books. Education is best found in books". It would create two vectors one for "He loved books" and other for "Education is best found in books." It would treat both of them orthogonal which makes them independent, but in reality, they are related to each other
To overcome these limitation word embedding was developed and word2vec is an approach to implement such.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer=CountVectorizer()

In [ ]:
x=vectorizer.fit_transform(text3)

In [ ]:
vectorizer.get_feature_names()

In [ ]:
x=x.toarray()

In [ ]:
x.shape

In [ ]:
x.view

# Word Embedding

 Word Embedding is a type of word representation that allows words with similar meaning to be understood by machine learning algorithms. Technically speaking, it is a mapping of words into vectors of real numbers There are various word embedding models available such as word2vec (Google), Glove (Stanford) and fastest (Facebook).

https://www.guru99.com/word-embedding-word2vec.html#1# https://www.guru99.com/word-embedding-word2vec.html#1

#  produce word embedding for better word representation.

# (1) word2vec

word2vec is a two-layer network where there is input one hidden layer and output.



# Word2vec Architecture
There are two architectures used by word2vec
1.Continuous Bag of words (CBOW)
2.skip gram
And They takes the input and output as one hot encoding 

#  1.Skip gram:
predict the context words from the target word

In [ ]:
corpus = [x for x in text3 if not x in stop_words1]


In [ ]:
corpus

In [ ]:


words = set(corpus)

In [ ]:
words

In [ ]:
word2int = {}
for i,word in enumerate(words):
    word2int[word] = i

WINDOW_SIZE = 4

data = []

for idx, word in enumerate(corpus):
    for neighbor in corpus[max(idx - WINDOW_SIZE, 0) : min(idx + WINDOW_SIZE, len(corpus)) + 1] : 
         if neighbor != word:
                data.append([word, neighbor])

In [ ]:
df = pd.DataFrame(data, columns = ['input', 'label'])


In [ ]:
import pandas as pd
for text in corpus:
    print(text)

In [ ]:
df.head()

In [ ]:
df.shape


In [ ]:
word2int

# Define Tensorflow Graph¶


In [ ]:
import tensorflow as tf
import numpy as np

ONE_HOT_DIM = len(words)

# function to convert numbers to one hot vectors
def to_one_hot_encoding(data_point_index):
    one_hot_encoding = np.zeros(ONE_HOT_DIM)
    one_hot_encoding[data_point_index] = 1
    return one_hot_encoding

X = [] # input word
Y = [] # target word

for x, y in zip(df['input'], df['label']):
    X.append(to_one_hot_encoding(word2int[ x ]))
    Y.append(to_one_hot_encoding(word2int[ y ]))

# convert them to numpy arrays
X_train = np.asarray(X)
Y_train = np.asarray(Y)
# making placeholders for X_train and Y_train
x = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))
y_label = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))

# word embedding will be 2 dimension for 2d visualization
EMBEDDING_DIM = 2 

# hidden layer: which represents word vector eventually
W1 = tf.Variable(tf.random_normal([ONE_HOT_DIM, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([1])) #bias
hidden_layer = tf.add(tf.matmul(x,W1), b1)

# output layer
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, ONE_HOT_DIM]))
b2 = tf.Variable(tf.random_normal([1]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_layer, W2), b2))

# loss function: cross entropy
loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), axis=[1]))

# training operation
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(loss)


# Train

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) 

iteration = 1000000
for i in range(iteration):
    # input is X_train which is one hot encoded word
    # label is Y_train which is one hot encoded neighbor word
    sess.run(train_op, feed_dict={x: X_train, y_label: Y_train})
    if i % 3000 == 0:
        print('iteration '+str(i)+' loss is : ', sess.run(loss, feed_dict={x: X_train, y_label: Y_train}))

In [ ]:

# Now the hidden layer (W1 + b1) is actually the word look up table
vectors = sess.run(W1 + b1)
print(vectors)

# word vector in table¶


In [ ]:
w2v_df = pd.DataFrame(vectors, columns = ['x1', 'x2'])
w2v_df['word'] = words
w2v_df = w2v_df[['word', 'x1', 'x2']]
w2v_df

# word vector in 2d chart¶


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

for word, x1, x2 in zip(w2v_df['word'], w2v_df['x1'], w2v_df['x2']):
    ax.annotate(word, (x1,x2 ))
    
PADDING = 1.0
x_axis_min = np.amin(vectors, axis=0)[0] - PADDING
y_axis_min = np.amin(vectors, axis=0)[1] - PADDING
x_axis_max = np.amax(vectors, axis=0)[0] + PADDING
y_axis_max = np.amax(vectors, axis=0)[1] + PADDING
 
plt.xlim(x_axis_min,x_axis_max)
plt.ylim(y_axis_min,y_axis_max)
plt.rcParams["figure.figsize"] = (30,40)

plt.show()

#  2.Continuous Bag of words (CBOW): 
predict the target word from the Context